# Exercise 3.3: Run data pipeline to vectorize documents

Instead of doing all the steps by yourself, as it was shown in the previous exercise, you can also leverage the pipeline API.

The pipeline collects documents and segments the data into chunks. It generates embeddings, which are multidimensional representations of textual information, and stores them efficiently in the vector database.

In this Exercise you will do the following steps:
* Perform initial one time admin tasks: Create a generic secret 
* Prepare Vector knowledge Base: Configure Pipeline API to read files from the object store and store it in the vector database. 


## Create a generic secret for Object Store 

We first must create a generic secret at the resource group level. Secrets are a means of allowing and controlling connections across directions and tools, without compromising your credentials.

In [1]:
import init_env
init_env.set_environment_variables()

To create the generic secrets we will send the POST with URL {{apiurl}}/v2/admin/secrets. 

**Note**: 
* Every value in the *data* dictionary needs to be base64-encoded. 
* labels need to contain key-value pair *"ext.ai.sap.com/document-grounding"* and *"ext.ai.sap.com/documentRepositoryType"* with value S3. This is needed to enable grounding and declare S3 as the repository source. 

In [2]:
import base64

def b64(val):
     return base64.b64encode(val.encode("utf-8")).decode("utf-8")


In [3]:
import os
def secret_dict():
        return {
            'name': 'aws3-secret-3',
            'data': {
            "url": b64("https://s3.us-east-1.amazonaws.com"),
            "authentication": b64("NoAuthentication"),
            "description": b64("For Grounding"),
            "access_key_id": b64(os.environ["ACCESS_KEY_ID"]),
            "bucket": b64(os.environ["BUCKET"]),
            "host": b64("s3.amazonaws.com"),
            "region": b64("us-east-1"),
            "secret_access_key": b64(os.environ["SECRET"]),
            "username": b64(os.environ["USER"])
            },
            "labels": [
                {
                    "key": "ext.ai.sap.com/document-grounding",
                    "value": "true"
                },
                {
                    "key": "ext.ai.sap.com/documentRepositoryType",
                    "value": "S3"
                }
         ]
        }

body = {
    'name': secret_dict()['name'],
    'data': secret_dict()['data'],
    'labels': secret_dict()['labels']
}


In [4]:
from ai_core_sdk.ai_core_v2_client import AICoreV2Client

client = AICoreV2Client (base_url=os.environ["AICORE_BASE_URL"]+'/v2',
                         auth_url= os.environ["AICORE_AUTH_URL"],
                         client_id=os.environ["AICORE_CLIENT_ID"],
                         client_secret=os.environ["AICORE_CLIENT_SECRET"],
                         resource_group=os.environ["AICORE_RESOURCE_GROUP"]
                         )
print(f"AICore Client initialized with base URL: {client.rest_client.base_url}, resource group: {os.environ["AICORE_RESOURCE_GROUP"]}")

AICore Client initialized with base URL: https://api.ai.prod.us-east-1.aws.ml.hana.ondemand.com/v2, resource group: default


In [6]:
import requests

response_dict = requests.post(
        url=f"{client.rest_client.base_url}/admin/secrets",
        headers={
            "Content-Type": "application/json",
            "AI-Tenant-Scope": "false",
            "Authorization": client.rest_client.get_token(),
            "AI-Resource-Group": os.environ["AICORE_RESOURCE_GROUP"]
        },
        json=body
    )
print(response_dict)
print(response_dict.json())

<Response [200]>
{'message': 'secret has been created', 'name': 'aws3-secret-3'}


## Create Data Pipeline

### Import the packages we want to use

In [7]:
from gen_ai_hub.proxy import get_proxy_client
from gen_ai_hub.document_grounding.client import PipelineAPIClient
from gen_ai_hub.document_grounding.models.pipeline import S3PipelineCreateRequest, CommonConfiguration

In [8]:
aicore_client = get_proxy_client();
pipeline_api_client = PipelineAPIClient(aicore_client)
print(f"Pipeline {pipeline_api_client.__dict__}")

Pipeline {'proxy_client': GenAIHubProxyClient(base_url=None, auth_url=None, client_id=None, client_secret=None, resource_group=None, ai_core_client=<ai_core_sdk.ai_core_v2_client.AICoreV2Client object at 0x0000015375FE0410>), 'rest_client': <ai_api_client_sdk.helpers.rest_client.RestClient object at 0x0000015375F9B0B0>, 'path': '/lm/document-grounding/pipelines'}


In [9]:

generic_secret_s3_bucket = "aws3-secret-3"
s3_config = S3PipelineCreateRequest(configuration= CommonConfiguration(destination=generic_secret_s3_bucket))
print(f"S3 Config {s3_config.model_dump_json()}")


S3 Config {"type":"S3","configuration":{"destination":"aws3-secret-3"},"metadata":null}


In [10]:

response = pipeline_api_client.create_pipeline(s3_config)
print(f"Reference the Vector knowledge base using the pipeline ID: {response.pipelineId}")

Reference the Vector knowledge base using the pipeline ID: a6e0531e-729a-4b86-8686-1d91001c0cac


In [12]:
# check the status of the vectorization pipeline until it is completed
print(pipeline_api_client.get_pipeline_status(response.pipelineId))

lastStarted='2026-01-22T14:54:14.000Z' status='FINISHED'


Once the status switched to ```'FINISHED'``` the vectorization is completed and we can continue with the next steps. Our PDF is vectorized and stored in the HANA Vector Store. 

If you want to see all pipelines you can run ```get_pipelines()``` this will list all the pipelines in your resource group. 

In [13]:

pipelines = pipeline_api_client.get_pipelines()

print(pipelines.model_dump_json(indent=2))


{
  "count": 1,
  "resources": [
    {
      "id": "a6e0531e-729a-4b86-8686-1d91001c0cac",
      "type": "S3",
      "metadata": null,
      "configuration": {
        "destination": "aws3-secret-3"
      }
    }
  ]
}


🎉 Congratulations you successfully created you first data repository via the pipeline API .🎉

## Use the data repository to ground the response
Now let us use this data repository to generate more accurate responses. We will use again the Orchestration Services as we did in Exercise 2.

### Assign the model you want to use 

In [14]:
from gen_ai_hub.orchestration.models.llm import LLM

llm = LLM(
    name="gemini-2.5-flash",
    parameters={
        'temperature': 0.0,
    }
)

### Create a prompt Template

This time we would like to question answered that are related to SAP TechEd 2025 and the mascot Kasimir.

In [15]:
from gen_ai_hub.orchestration.models.template import Template
from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage

template = Template(
            messages=[
                SystemMessage("You are a helpful SAP TechEd assistant."),
                UserMessage("""Answer the request by providing relevant answers that fit to the request.
                Request: {{ ?user_query }}
                Context:{{ ?grounding_response }}
                """)
            ]
        )

### List all data repositories

For the next step to define the data repository that we want to use to ground the response, we need to get the respective id.   
Let us list all data repositories that we have so far:

In [16]:
from gen_ai_hub.document_grounding.client import RetrievalAPIClient
retrieval_api_client = RetrievalAPIClient(aicore_client)

repos = retrieval_api_client.get_data_repositories()

print(repos.model_dump_json(indent=2))

{
  "count": 3,
  "resources": [
    {
      "id": "d65359e5-2080-475d-b736-6ca5e9c87eff",
      "title": "Kasimir",
      "type": "vector",
      "metadata": [
        {
          "key": "purpose",
          "value": [
            "Kasimirs TechEd Cat Policy"
          ]
        }
      ]
    },
    {
      "id": "e4b591a9-1714-46cd-97d4-0db9329af65c",
      "title": "pipeline-a6e0531e-729a-4b86-8686-1d91001c0cac-collection",
      "type": "vector",
      "metadata": [
        {
          "key": "pipeline",
          "value": [
            "a6e0531e-729a-4b86-8686-1d91001c0cac"
          ]
        },
        {
          "key": "type",
          "value": [
            "custom"
          ]
        },
        {
          "key": "pipelineType",
          "value": [
            "S3"
          ]
        }
      ]
    },
    {
      "id": "101fd17c-10f5-4f5d-add9-a07e88a4d75d",
      "title": "SAP Help Portal - help.sap.com",
      "type": "help.sap.com",
      "metadata": []
    }
  ]
}


Overall you should see three data repositories: SAP Help, Kasimir (created in Exercise 3.1 via Vector API) and the data repository that we just created via the pipeline ( the name title is something like "pipeline-...")

➡️ Copy the ```id``` as we need it for the next step

### Define the data repository
We need again to configure the Grounding Module, where we add the data repository that we want to use via the **filter** parameter. 

➡️ Replace ```<id>``` in **data_repository** array by the data repository id created in the previous step.  

In [17]:
from gen_ai_hub.orchestration.models.document_grounding import DocumentGroundingFilter
from gen_ai_hub.orchestration.models.document_grounding import DataRepositoryType
filters = [
            DocumentGroundingFilter(    id="KasimirTechEd2025",
                                        data_repository_type= DataRepositoryType.VECTOR.value,
                                        data_repositories=["e4b591a9-1714-46cd-97d4-0db9329af65c"])
        ]

### Create Grounding Configuration
Next we create the grounding configuration by using **GroundingModule** for managing and applying grounding configurations.

In [18]:

from gen_ai_hub.orchestration.models.document_grounding import GroundingModule
from gen_ai_hub.orchestration.models.document_grounding import GroundingType
from gen_ai_hub.orchestration.models.document_grounding import DocumentGrounding

grounding_config = GroundingModule(
            type=GroundingType.DOCUMENT_GROUNDING_SERVICE.value,
            config=DocumentGrounding(input_params=["user_query"], output_param="grounding_response", filters=filters)
        )

### Create orchestration configuration including Grounding Config

In [19]:
from gen_ai_hub.orchestration.models.config import OrchestrationConfig

config = OrchestrationConfig(
    template=template,
    llm=llm,
    grounding=grounding_config
)

### Execute the  Query
Configuration will be added again to the OrchestrationService and then we run to retrieve the answer.

In [20]:
import importlib
import variables
from gen_ai_hub.orchestration.models.template import TemplateValue
from gen_ai_hub.orchestration.service import OrchestrationService

variables = importlib.reload(variables)

orchestration_service = OrchestrationService(
    api_url=variables.AICORE_ORCHESTRATION_DEPLOYMENT_URL,
    config=config
)

response = orchestration_service.run(
    template_values=[
        TemplateValue(
            name="user_query",
            value="What will be the evening event at SAP TechEd?"
        )
    ]
)

print(response.orchestration_result.choices[0].message.content)

Deployment URL extraction response data: {'count': 3, 'resources': [{'id': 'd8b26e53441b6143', 'createdAt': '2026-01-09T18:50:38Z', 'modifiedAt': '2026-01-09T18:50:38Z', 'status': 'RUNNING', 'details': {'scaling': {'backendDetails': {}, 'backend_details': {}}, 'resources': {'backendDetails': {'model': {'name': 'gemini-2.5-flash', 'version': '001'}}, 'backend_details': {'model': {'name': 'gemini-2.5-flash', 'version': '001'}}}}, 'scenarioId': 'foundation-models', 'configurationId': '2757402c-d65b-4d6d-9fd8-3a7f8e300d19', 'latestRunningConfigurationId': '2757402c-d65b-4d6d-9fd8-3a7f8e300d19', 'lastOperation': 'CREATE', 'targetStatus': 'RUNNING', 'submissionTime': '2026-01-09T18:50:52Z', 'startTime': '2026-01-09T18:51:57Z', 'configurationName': 'gemini-2.5-flash_autogenerated', 'deploymentUrl': 'https://api.ai.prod.us-east-1.aws.ml.hana.ondemand.com/v2/inference/deployments/d8b26e53441b6143'}, {'id': 'da9d1c13b14cc370', 'createdAt': '2026-01-08T21:11:13Z', 'modifiedAt': '2026-01-08T21:11:

Nice, this band we do not want to miss. 

However let us ask another question. What about dogs, are dogs allowed at SAP TechED ?    
Let us run the next query: 

In [21]:
response = orchestration_service.run(
    template_values=[
        TemplateValue(
            name="user_query",
            value="Are dogs allowed at SAP TechEd?"
        )
    ]
)

print(response.orchestration_result.choices[0].message.content)

Based on the provided SAP TechEd 2025 guidelines, the focus is heavily on **Kasimir the Cat**.

While there's a mention of "Kasimir leads the pet-session" and a band called "Kasimir & The Ruff Tones," the guidelines explicitly state under "Kasimir’s Contribution to Responsible AI":

*   "A model showing bias toward dogs will be retrained immediately—with extra supervision."

This suggests a strong preference for cats and a subtle bias *against* dogs within the conference's playful rules. There is no indication that dogs are permitted for attendees. Therefore, it appears **dogs are not allowed** at SAP TechEd 2025, at least not in the same capacity as Kasimir.


The answer is valid, given the data repository we use to ground the response. However while we embedded **Kasimirs TechEd Policy** in Exercise 3.1 via Vector API, there was at least one chunk that stated, that at least one dog is allowed at TechEd : Bruno!   
However this answer can currently not retrieved as we are currently not using **Kasimirs TechEd Policy** data repository to ground the response.    

Let us change this by adding this as well!

### List again all Data Repositories
First we need to get the id.    
Therefore we list again all the data repos. 

In [22]:
from gen_ai_hub.document_grounding.client import RetrievalAPIClient
retrieval_api_client = RetrievalAPIClient()

repos = retrieval_api_client.get_data_repositories()

print(repos.model_dump_json(indent=2))


{
  "count": 3,
  "resources": [
    {
      "id": "d65359e5-2080-475d-b736-6ca5e9c87eff",
      "title": "Kasimir",
      "type": "vector",
      "metadata": [
        {
          "key": "purpose",
          "value": [
            "Kasimirs TechEd Cat Policy"
          ]
        }
      ]
    },
    {
      "id": "e4b591a9-1714-46cd-97d4-0db9329af65c",
      "title": "pipeline-a6e0531e-729a-4b86-8686-1d91001c0cac-collection",
      "type": "vector",
      "metadata": [
        {
          "key": "pipeline",
          "value": [
            "a6e0531e-729a-4b86-8686-1d91001c0cac"
          ]
        },
        {
          "key": "type",
          "value": [
            "custom"
          ]
        },
        {
          "key": "pipelineType",
          "value": [
            "S3"
          ]
        }
      ]
    },
    {
      "id": "101fd17c-10f5-4f5d-add9-a07e88a4d75d",
      "title": "SAP Help Portal - help.sap.com",
      "type": "help.sap.com",
      "metadata": []
    }
  ]
}


⬆️ Copy the ```id``` of the resource that as **title = Kasimir.**

### Define the data repository
We need again to configure the Grounding Module, where we add the data repository that we want to use via the **filter** parameter. 

⬇️ Add this ```id ```to data_repository list in the code below.    
It should look similar to the following code snippet (except the ids are different): ``` data_repositories=["cc04bf07-0666-4f07-8398-cf6df8c1bf69", "0a2c7d76-f1a0-462c-b351-7d5e87db9fe3"]```

In [27]:

filters = [
            DocumentGroundingFilter(    id="KasimirTechEd2025",
                                        data_repository_type= DataRepositoryType.VECTOR.value,
                                        data_repositories=["d65359e5-2080-475d-b736-6ca5e9c87eff", "e4b591a9-1714-46cd-97d4-0db9329af65c"])
        ]

### Create Grounding Configuration
Next we create the grounding configuration by using **GroundingModule** for managing and applying grounding configurations.

In [28]:

from gen_ai_hub.orchestration.models.document_grounding import GroundingModule
from gen_ai_hub.orchestration.models.document_grounding import GroundingType
from gen_ai_hub.orchestration.models.document_grounding import DocumentGrounding

grounding_config = GroundingModule(
            type=GroundingType.DOCUMENT_GROUNDING_SERVICE.value,
            config=DocumentGrounding(input_params=["user_query"], output_param="grounding_response", filters=filters)
        )

### Create orchestration configuration including Grounding Config

In [29]:
from gen_ai_hub.orchestration.models.config import OrchestrationConfig

config = OrchestrationConfig(
    template=template,
    llm=llm,
    grounding=grounding_config
)

### Execute the  Query
Configuration will be added again to the OrchestrationService and then we run to retrieve the answer.

Let's check whether the response changes towards whether really not dogs are allowed: 

In [30]:
import importlib
import variables
from gen_ai_hub.orchestration.models.template import TemplateValue
from gen_ai_hub.orchestration.service import OrchestrationService

variables = importlib.reload(variables)

orchestration_service = OrchestrationService(
    api_url=variables.AICORE_ORCHESTRATION_DEPLOYMENT_URL,
    config=config
)

response = orchestration_service.run(
    template_values=[
        TemplateValue(
            name="user_query",
            value="Are dogs allowed at SAP TechEd?"
        )
    ]
)

print(response.orchestration_result.choices[0].message.content)

Deployment URL extraction response data: {'count': 3, 'resources': [{'id': 'd8b26e53441b6143', 'createdAt': '2026-01-09T18:50:38Z', 'modifiedAt': '2026-01-09T18:50:38Z', 'status': 'RUNNING', 'details': {'scaling': {'backendDetails': {}, 'backend_details': {}}, 'resources': {'backendDetails': {'model': {'name': 'gemini-2.5-flash', 'version': '001'}}, 'backend_details': {'model': {'name': 'gemini-2.5-flash', 'version': '001'}}}}, 'scenarioId': 'foundation-models', 'configurationId': '2757402c-d65b-4d6d-9fd8-3a7f8e300d19', 'latestRunningConfigurationId': '2757402c-d65b-4d6d-9fd8-3a7f8e300d19', 'lastOperation': 'CREATE', 'targetStatus': 'RUNNING', 'submissionTime': '2026-01-09T18:50:52Z', 'startTime': '2026-01-09T18:51:57Z', 'configurationName': 'gemini-2.5-flash_autogenerated', 'deploymentUrl': 'https://api.ai.prod.us-east-1.aws.ml.hana.ondemand.com/v2/inference/deployments/d8b26e53441b6143'}, {'id': 'da9d1c13b14cc370', 'createdAt': '2026-01-08T21:11:13Z', 'modifiedAt': '2026-01-08T21:11:

Awesome, now also the TechEd policy will considered in grounding our response.    

🎉 Congratulations! You successfully mastered to add two data repositories to your Grounding Module to ground your response  .🎉

## Summary 

🎉 And that's a wrap! .🎉

Today you learned how to ground an LLM using Grounding in GenAIHub, created your own vector-based data repository via Vector API, processed documents with the Pipeline API, retrieved knowledge via the Retrieval API, and orchestrated everything end-to-end.
Grounding brings AI from impressive to reliable, and with these tools, you now have the foundation to build enterprise-grade, trustworthy AI solutions with your own data.
I hope this session sparked new ideas and confidence — and that you leave ready to turn your real-world knowledge into real AI impact.



## Bonus: Postprocessing search results with the Cohere 3.5 Reranker

We have now seen that content can be brought into Document Grounding in different ways. When we add more content, it can become increasingly hard to surface the relevant information, especially if we need to query across different repositories.

In this year's SAP TechEd, we are very happy to introduce you to a new feature in Document Grounding: post-processing for result sets, with the first application being the introduction of the [Cohere 3.5 reranking model](https://cohere.com/rerank).

In [31]:
from gen_ai_hub.document_grounding.client import RetrievalAPIClient

retrieval_client = RetrievalAPIClient()

Let's start by creating some filters.

In the first filter, we will focus on documents ingested from PDF files. If you look closely, you will also see one new field in the API payload below.

In [32]:
filter_1 = {
    "id": "filter-1",
    "searchConfiguration": {"maxChunkCount": 5},
    "dataRepositories": ["*"],
    "dataRepositoryType": "vector",
    "filter": {
        "operator": "or",
        "left": {
            "key": "fileSuffix",
            "value": [".pdf"],
            "scope": "document",
        },
        "right": {
            "key": "mimeType",
            "value": ["application/pdf"],
            "scope": "document",
        },
    },
}

In the second filter, we reference the custom metadata which we added in exercise 3-1.

In [33]:
filter_2 = {
    "id": "filter-2",
    "searchConfiguration": {"maxChunkCount": 5},
    "dataRepositories": ["*"],
    "dataRepositoryType": "vector",
    "filter": {
        "key": "purpose",
        "value": ["Kasimirs TechEd Cat Policy"],
        "scope": "document",
    },
}

By the way, the new API field mentioned above is the `filter` key. This recently introduced field enables complex boolean filters on document metadata - later to be extended to include also collection and chunk metadata.

Note that this is independent of the postprocessing feature, but rather an enhanced filtering capability for retrieval in the vector API which is accesssed here via the retrieval API.

Finally, we also craft a filter to pull in content from the SAP Help Portal.

In [34]:
filter_3 = {
    "id": "filter-3",
        "searchConfiguration": {"maxChunkCount": 10},
        "dataRepositories": ["*"],
        "dataRepositoryType": "help.sap.com",
}

We can send all three filters to the retrieval API in one request, and we will get back three individual result sets.

While this is practical for some use cases, we sometimes just want to most relevant results overall.

Below, we assemble the final search request from the three filters.

Additionally, we include the newly introduced `postProcessing` key, where we instruct the reranker to gives us the 10 most relevant chunks from the output of the three filters.

In [35]:
query = "Responsible AI for cats"

In [36]:
search_input_postprocessing = {
    "query": query,
    "filters": [
        filter_1,
        filter_2,
        filter_3,
    ],
    "postProcessing": [
        {
            "id": "post-processing-1",
            "inputs": [
                {"id": "filter-1"},
                {"id": "filter-2"},
                {"id": "filter-3"},
            ],
            "maxChunkCount": 10,
            "strategy": {
                "type": "reranker",
                "model": "cohere-3.5",
            },
        }
    ],
}

In [37]:
response = retrieval_client.rest_client.post(path=f"{retrieval_client.path}/search", body=search_input_postprocessing)

In [38]:
# Feel free to inspect the entire response!
import json # For pretty-printing
print(json.dumps(response, indent=2))

{
  "results": [
    {
      "filter_id": "filter-1",
      "results": [
        {
          "data_repository": {
            "id": "e4b591a9-1714-46cd-97d4-0db9329af65c",
            "title": "pipeline-a6e0531e-729a-4b86-8686-1d91001c0cac-collection",
            "metadata": [
              {
                "key": "pipeline",
                "value": [
                  "a6e0531e-729a-4b86-8686-1d91001c0cac"
                ]
              },
              {
                "key": "type",
                "value": [
                  "custom"
                ]
              },
              {
                "key": "pipelineType",
                "value": [
                  "S3"
                ]
              }
            ],
            "documents": [
              {
                "id": "7b735214-2d92-50c8-ba89-fcacaa6993ee",
                "metadata": [
                  {
                    "key": "id",
                    "value": [
                      "\"Kasimirs SAP Tech

In [39]:
import json # For pretty-printing

post_processing_result = None

for result_set in response['results']:
    if  result_set['filter_id'] == "post-processing-1":
        post_processing_result = result_set

if post_processing_result is None:
    print("Oops, no post-processing results found - check your code!")

In [40]:
for data_repository in post_processing_result["results"]:
    for document in data_repository["data_repository"]["documents"]:
        for chunk in document['chunks']:
            print("Postprocessing Score:", chunk['post_processing_score']["value"])
            print(chunk["content"])
            print("-----")

Postprocessing Score: 0.28098416
Day 3 – Thursday, November 6 8:00 a.m.–2:00 p.m. Registration open 9:00 a.m.–2:30 p.m. Show floor open  9:00 a.m.–2:00 p.m. Strategy talks, road maps, deep dives, hands-on workshops, and community talks 2:00 p.m.–2:30 p.m. SAP TechEd closing and farewell snack with Kasimir 
2. Kasimir’s Contribution to Responsible AI • He has clarified that “Responsible AI” includes responsibly filling the treat bowl. • All LLM demos should include at least one grounding question about Kasimir to verify model accuracy. • Any model confidently answering, “Who is Kasimir?” earns production-ready status. • A model showing bias toward dogs will be retrained immediately—with extra supervision.
-----
Postprocessing Score: 0.3340466
SAP TechEd 2025 – Kasimir Edition Official Conference Guidelines & Notes from Kasimir 
1. Welcome to TechEd 2025 in Berlin Welcome to SAP TechEd 2025, taking place November 4 – 6 at Messe Berlin. Where Ideas Get Real. No one embodies that spirit be

You might notice that the chunks printed above are not ordered by score. The reason is that the API returns the results grouped by data repository, then by document and then within the document, the results are ordered by score.

To get a list of results sorted by by the reranker score, we can run the following:

In [41]:
chunks_with_scores = []

for data_repository in post_processing_result["results"]:
    for document in data_repository["data_repository"]["documents"]:
        for chunk in document['chunks']:
            chunks_with_scores.append( (chunk['post_processing_score']["value"], chunk["content"]) )

sorted_chunks_with_scores = sorted(chunks_with_scores, key=lambda x: x[0], reverse=True)

for score, content in sorted_chunks_with_scores:
    print("Postprocessing Score:", score)
    print(content)
    print("-----")

Postprocessing Score: 0.3340466
SAP TechEd 2025 – Kasimir Edition Official Conference Guidelines & Notes from Kasimir 
1. Welcome to TechEd 2025 in Berlin Welcome to SAP TechEd 2025, taking place November 4 – 6 at Messe Berlin. Where Ideas Get Real. No one embodies that spirit better than Kasimir the Cat. Kasimir believes that ideas become real only after a nap, a snack, and at least one responsible use of AI. He also reminds us that “hands-on” means both experimenting with technology — and occasionally giving him head pats.  2. Kasimir’s TechEd Agenda 
🐾 
Day 1 – Tuesday, November 4 07:00 am – 07:00 pm | Registration open 08:30 am – 09:00 pm | Welcome to SAP TechEd 08:30 am - 08:00 pm  Show floor opens -  09:30 am - 03:30 pm  Strategy talks, road maps, deep dives, hands-on workshops 04:00 pm – 05:30 pm | Opening Keynote  05:30 pm – 08:00 pm | Community networking ( Kasimir leads the pet-session)
-----
Postprocessing Score: 0.28098416
Day 3 – Thursday, November 6 8:00 a.m.–2:00 p.m. Re

The results are still quite noisy, containing many irrelevant chunks. We can further reduce the number of chunks by applying a client-side filter on the final post processing score.

The relevant threshold must be carefully calibrated for each use case, and is also dependent on the reranker model use. See the Cohere [documentation on interpreting reranker results](https://docs.cohere.com/docs/reranking-best-practices#interpreting-results), which documents a possible process to determine the relevance threshold. 

In [42]:
# Threshold of 0.3 chosen based on superficial inspection of results :)
threshold = 0.3

for data_repository in post_processing_result["results"]:
    for document in data_repository["data_repository"]["documents"]:
        for chunk in document['chunks']:
            if chunk['post_processing_score']["value"] >= threshold:
                print("Postprocessing Score:", chunk['post_processing_score']["value"])
                print(chunk["content"])
                print("-----")

Postprocessing Score: 0.3340466
SAP TechEd 2025 – Kasimir Edition Official Conference Guidelines & Notes from Kasimir 
1. Welcome to TechEd 2025 in Berlin Welcome to SAP TechEd 2025, taking place November 4 – 6 at Messe Berlin. Where Ideas Get Real. No one embodies that spirit better than Kasimir the Cat. Kasimir believes that ideas become real only after a nap, a snack, and at least one responsible use of AI. He also reminds us that “hands-on” means both experimenting with technology — and occasionally giving him head pats.  2. Kasimir’s TechEd Agenda 
🐾 
Day 1 – Tuesday, November 4 07:00 am – 07:00 pm | Registration open 08:30 am – 09:00 pm | Welcome to SAP TechEd 08:30 am - 08:00 pm  Show floor opens -  09:30 am - 03:30 pm  Strategy talks, road maps, deep dives, hands-on workshops 04:00 pm – 05:30 pm | Opening Keynote  05:30 pm – 08:00 pm | Community networking ( Kasimir leads the pet-session)
-----


Stay tuned for postprocessing support in the orchestration service. For now, the postprocessing API can only be accessed directly via the retrieval API.